In [7]:
# Install the surprise package
!pip install -q -U scikit-surprise
from surprise import Dataset, Reader
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import arff
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise import SVD
import random
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [90]:
path = os.path.expanduser('~/Documents/Studium/Master/Web Mining/Project/data_kindle_preprocessed.xlsx')
data_preprocessed = pd.read_excel(path, index_col=[0])

In [91]:
data = data_preprocessed.copy()

In [92]:
# Replace non-finite values with NaN
data['publication_year'] = data['publication_year'].replace([np.inf, -np.inf], np.nan)

# Convert NaN to a placeholder value (e.g., -1)
data['publication_year'].fillna(-1, inplace=True)

# Convert the column to integers
data['publication_year'] = data['publication_year'].astype(int).astype(str)

In [93]:
data

,rating,reviewerID,asin,title,brand,language,print_length_category,publication_year,category_string,paid_free
0,1.0,A3OC8ZG1S3OAVA,B0015Z7VFQ,Look What Santa Brought (The Perfect Gift) - K...,Visit Amazon's Annmarie McKenna Page,English,small,2009,"Kindle Store, Kindle eBooks, Literature & Fiction",Paid
1,4.0,A2U8YWPP1PYHJM,B0017HNV1U,Babylonian Laws- The Oldest Code of Laws in th...,King of Babylon Hammurabi,English,small,2008,"Kindle Store, Kindle eBooks, History",Paid
2,3.0,A3361XGKYF17S3,B001892EI8,The Billionaire&s Baby (Harlequin Mini # 19) -...,Leanne Banks,English,small,2008,"Kindle Store, Kindle eBooks, Romance",Paid
3,4.0,AVGYENZU56KBR,B001892EI8,The Billionaire&s Baby (Harlequin Mini # 19) -...,Leanne Banks,English,small,2008,"Kindle Store, Kindle eBooks, Romance",Paid
4,3.0,A3361XGKYF17S3,B001892DGG,The Wallflower (Halle Puma Book 1) - Kindle ed...,Visit Amazon's Dana Marie Bell Page,English,small,2008,"Kindle Store, Kindle eBooks, Romance",Paid
...,...,...,...,...,...,...,...,...,...,...
19566,4.0,A1EQY74OFGE4NE,B01HIGNUGE,Final Jackpot - Kindle edition,Visit Amazon's Parker Avrile Page,English,small,2016,"Kindle Store, Kindle eBooks, Literature & Fiction",Paid
19567,3.0,A1EQY74OFGE4NE,B01HINH1WQ,Claimed,Visit Amazon's Wolf Specter Page,English,medium,2016,"Kindle Store, Kindle eBooks, Literature & Fiction",Paid
19568,5.0,A1SVA69J57MX2A,B01HIOR0S0,Battle Beyond Earth,Visit Amazon's Nick S. Thomas Page,English,small,2016,"Kindle Store, Kindle eBooks, Science Fiction &...",Paid
19569,5.0,A3FVMG7SWNF7QR,B01HIULQXY,Clarity&s Doom (Ancient Origins Book 1) - Kind...,Visit Amazon's C.L. Scholey Page,English,small,2016,"Kindle Store, Kindle eBooks, Literature & Fiction",Paid


In [94]:
data["book_info"] =  data['category_string'] + '  ' + data['brand'] + '  ' + data['paid_free']+ ' ' + data['print_length_category'] + ' ' + data['publication_year'] + '  ' + data['language'] 
data.drop(['rating', 'brand', 'reviewerID', 'language','print_length_category', 'publication_year', 'category_string', 'paid_free'],axis=1,inplace=True)
data.drop_duplicates(subset=['asin', 'title'], inplace=True)
data.head()

,asin,title,book_info
0,B0015Z7VFQ,Look What Santa Brought (The Perfect Gift) - K...,"Kindle Store, Kindle eBooks, Literature & Fict..."
1,B0017HNV1U,Babylonian Laws- The Oldest Code of Laws in th...,"Kindle Store, Kindle eBooks, History King of ..."
2,B001892EI8,The Billionaire&s Baby (Harlequin Mini # 19) -...,"Kindle Store, Kindle eBooks, Romance Leanne B..."
4,B001892DGG,The Wallflower (Halle Puma Book 1) - Kindle ed...,"Kindle Store, Kindle eBooks, Romance Visit Am..."
5,B001BRD238,Secrets: a PsyCop Novel - Kindle edition,"Kindle Store, Kindle eBooks, Romance Visit Am..."


In [95]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Vectorize the text data using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['book_info'])

# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

# Debugging print
print("Train Data Shape:", train_data.shape)
print("Validation Data Shape:", val_data.shape)
print("Test Data Shape:", test_data.shape)

# Calculate cosine similarity matrices
tfidf_matrix_train = tfidf_vectorizer.transform(train_data['book_info'])
tfidf_matrix_val = tfidf_vectorizer.transform(val_data['book_info'])
tfidf_matrix_test = tfidf_vectorizer.transform(test_data['book_info'])

# Calculate cosine similarity matrices with correct dimensions
cosine_sim_train = cosine_similarity(tfidf_matrix_train, tfidf_matrix_train)
cosine_sim_val = cosine_similarity(tfidf_matrix_val, tfidf_matrix_train)
cosine_sim_test = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)

# Debugging print
print("Cosine similarity matrix size (Train):", cosine_sim_train.shape)
print("Cosine similarity matrix size (Validation):", cosine_sim_val.shape)
print("Cosine similarity matrix size (Test):", cosine_sim_test.shape)

# Implement function to recommend books
def recommend(title, cosine_sim_matrix, train_data):
    indices = train_data[train_data['title'] == title].index.tolist()
    if not indices:
        return []
    
    recommended_books = []
    for idx in indices:
        try:
            sim_scores = list(enumerate(cosine_sim_matrix[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]  # Top 10 similar items
            book_indices = [i[0] for i in sim_scores]
            recommended_books.extend(train_data['title'].iloc[book_indices].tolist())
        except IndexError:
            continue
    
    return recommended_books if recommended_books else []  # Return empty list if no recommendations found

# Pre-calculate recommendations for all books where recommendations are available
all_titles = train_data['title'].unique()
pred_content_based_recommender_system = {title: recommend(title, cosine_sim_train, train_data)[:10] for title in all_titles}

# Evaluate the recommender system
def evaluate_recommender(test_data, cosine_sim_matrix, train_data):
    y_true = []
    y_pred = []
    for title in test_data['title']:
        true_books = set(test_data[test_data['title'] == title]['title'])
        recommended_books = set(pred_content_based_recommender_system.get(title, []))  # Get precalculated recommendations
        y_true.append(len(true_books))
        y_pred.append(len(recommended_books))

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

# Evaluate the recommender system
mae_val, rmse_val = evaluate_recommender(val_data, cosine_sim_val, train_data)
mae_test, rmse_test = evaluate_recommender(test_data, cosine_sim_test, train_data)

print("Validation MAE:", mae_val)
print("Validation RMSE:", rmse_val)
print("Test MAE:", mae_test)
print("Test RMSE:", rmse_test)


Train Data Shape: (8700, 3)
Validation Data Shape: (2900, 3)
Test Data Shape: (2901, 3)
Cosine similarity matrix size (Train): (8700, 8700)
Cosine similarity matrix size (Validation): (2900, 8700)
Cosine similarity matrix size (Test): (2901, 8700)
Validation MAE: 1.0882758620689654
Validation RMSE: 1.3721365167830988
Test MAE: 1.0827300930713548
Test RMSE: 1.3517769530190797


In [96]:
pred_content_based_recommender_system.get('Second Touch (Emma&s Arabian Nights, #2) - Kindle edition')

['Spark (Ruin Outlaws MC series Book 2) - Kindle edition',
 'Candy Man - Kindle edition',
 'Christmas with the Billionaire (Holiday Encounters Book 1) - Kindle edition',
 ' Make-Believe Wife (Destiny Bay Romances',
 'The Apprenticeship of Julian St. Albans (Consulting Magic Book 2) - Kindle edition',
 'Turkey in the Snow - Kindle edition',
 'Puppy, Car, and Snow - Kindle edition',
 'A Brand New Step: A Taboo Short (Sexy Household Secrets: Man of the House Book 2) - Kindle edition',
 'Bitter Taffy (Candy Man Book 2) - Kindle edition',
 'Getting Lucky with the Rock Star (Holiday Encounters Book 3) - Kindle edition']

In [97]:
# vectorizing the book info column using TFidf Vectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


tf = TfidfVectorizer(analyzer="word", ngram_range=(1,2), min_df=1, stop_words='english')

tfidf_matrix = tf.fit_transform(data['book_info'])

cosine_sim =  cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

indices = pd.Series(data['title'])
indices[:5]

def recommend(title, cosine_sim = cosine_sim):
    if title not in indices.values:
        return "Title not found in the database."
    recommended_books = []
    idx = indices[indices == title].index[0]   # to get the index of book name matching the input book_name
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)   # similarity scores in descending order
    top_10_indices = list(score_series.iloc[1:11].index)   # to get the indices of top 10 most similar books
    # [1:11] to exclude 0 (index 0 is the input book itself)
    
    for i in top_10_indices:   # to append the titles of top 10 similar booksto the recommended_books list
        recommended_books.append(list(data['title'])[i])
        
    return recommended_books


#to output the recommendations.
recommend('Reagan&s Revenge and Ending Emily&s Engagement (The Reed Brothers Series) - Kindle edition')

[[1.         0.0190001  0.02312506 ... 0.04199093 0.0742936  0.03085186]
 [0.0190001  1.         0.26348392 ... 0.02212988 0.02601486 0.01829713]
 [0.02312506 0.26348392 1.         ... 0.02693433 0.03166275 0.02226948]
 ...
 [0.04199093 0.02212988 0.02693433 ... 1.         0.1441296  0.09686777]
 [0.0742936  0.02601486 0.03166275 ... 0.1441296  1.         0.11387327]
 [0.03085186 0.01829713 0.02226948 ... 0.09686777 0.11387327 1.        ]]


['Big Rock (Big Rock Book 1) - Kindle edition',
 'First Night: (Seductive Nights: Julia and Clay Book #0.5) - Kindle edition',
 'Every Second With You (No Regrets Book 2) - Kindle edition',
 'Caught Up in Her (Caught Up in Love) - Kindle edition',
 'The Start of Us (No Regrets) - Kindle edition',
 'Sinful Nights Bundle: Books 1-3 - Kindle edition',
 'Tempted By A Rogue - Kindle edition',
 'Blurred Lines (Love Unexpectedly) - Kindle edition',
 'Surviving For Us - Kindle edition',
 'ARENA (An Artemus Newton Thriller Book 2) - Kindle edition']

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Split data into train, validate, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Content Similarity
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(train_data["combined"])
cosine_similarities = linear_kernel(matrix, matrix)

indices = train_data.groupby('reviewerID').apply(lambda x: x.index.tolist())
indices = pd.Series(indices, index=indices.index)

#print(indices.head())

def content_recommender(reviewerID, train_data, vectorizer, cosine_similarities, indices):
    if reviewerID in indices:
        idx_list = indices[reviewerID]
        valid_indices = [idx for idx in idx_list if idx < cosine_similarities.shape[0]]
        sim_scores = []
        for idx in valid_indices:
            sim_scores.extend(enumerate(cosine_similarities[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:31]  # Top 30 similar books
        book_indices = [i[0] for i in sim_scores]
        return train_data.iloc[book_indices]
    else:
        print(f"ReviewerID {reviewerID} not found in the training data.")
        return None

# Example usage:
reviewerID = 'AVGYENZU56KBR'
recommended_books = content_recommender(reviewerID, train_data, vectorizer, cosine_similarities, indices)
if recommended_books is not None:
    print(recommended_books[['asin', 'combined']])
else:
    print("No recommendations available.")



# Evaluation
val_matrix = vectorizer.transform(val_data["combined"])
val_cosine_similarities = linear_kernel(val_matrix, matrix)

def predict_ratings(reviewerID, data, vectorizer, cosine_similarities):
    if reviewerID in indices:
        idx_list = indices[reviewerID]
        valid_indices = [idx for idx in idx_list if idx < cosine_similarities.shape[0]]
        sim_scores = [(i, cosine_similarities[i]) for i in valid_indices]
        
        print("sim_scores before sorting:", sim_scores)  # Add this line for debugging
        
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:31]  # Top 30 similar books
        book_indices = [i[0] for i in sim_scores]
        selected_books = data.iloc[book_indices]
        return selected_books
    else:
        print(f"ReviewerID {reviewerID} not found in the training data.")
        return None


def evaluate(reviewerID, data, vectorizer, cosine_similarities):
    predicted_books = predict_ratings(reviewerID, val_data, vectorizer, val_cosine_similarities)
    true_ratings = val_data[val_data['reviewerID'] == reviewerID]['rating']
    predicted_ratings = predicted_books['rating']
    mae = mean_absolute_error(true_ratings, predicted_ratings)
    rmse = mean_squared_error(true_ratings, predicted_ratings, squared=False)
    return mae, rmse

# Example usage:
mae, rmse = evaluate(reviewerID, val_data, vectorizer, val_cosine_similarities)
print(f'MAE: {mae}, RMSE: {rmse}')


KeyError: 'combined'

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Split data into train, validate, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Content Similarity
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(train_data["combined"])
cosine_similarities = linear_kernel(matrix, matrix)

indices = train_data.groupby('reviewerID').apply(lambda x: x.index.tolist())
indices = pd.Series(indices, index=indices.index)

#print(indices.head())

def content_recommender(train_data, vectorizer, cosine_similarities, indices):
    if reviewerID in indices:
        idx_list = indices[reviewerID]
        valid_indices = [idx for idx in idx_list if idx < cosine_similarities.shape[0]]
        sim_scores = []
        for idx in valid_indices:
            sim_scores.extend(enumerate(cosine_similarities[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:31]  # Top 10 similar books
        book_indices = [i[0] for i in sim_scores]

        print('train_data.iloc[book_indices]')
        print(train_data.iloc[book_indices])

        
        return train_data.iloc[book_indices]
    else:
        print(f"ReviewerID {reviewerID} not found in the training data.")
        return None

# Example usage:
reviewerID = 'AVGYENZU56KBR'
recommended_books = content_recommender(reviewerID, train_data, vectorizer, cosine_similarities, indices)
if recommended_books is not None:
    print(recommended_books[['asin', 'combined']])
else:
    print("No recommendations available.")



# Evaluation
val_matrix = vectorizer.transform(val_data["combined"])
val_cosine_similarities = linear_kernel(val_matrix, matrix)

def predict_ratings(reviewerID, data, vectorizer, cosine_similarities):

    print("Shape of cosine_similarities:", cosine_similarities.shape)  # Add this line for debugging   

    
    if reviewerID in indices:
        idx_list = indices[reviewerID]
        valid_indices = [idx for idx in idx_list if idx < cosine_similarities.shape[0]]
        sim_scores = [(i, cosine_similarities[i]) for i in valid_indices]
        
        print("sim_scores before sorting:", sim_scores)  # debugging

        print("Shape of cosine_similarities:", cosine_similarities.shape)  # Add this line for debugging

        sim_scores = sorted(sim_scores, key=lambda x: x[1][0], reverse=True)

        print("sim_scores after sorting:", sim_scores)  # Add this line for debugging

        sim_scores = sim_scores[1:31]  # Top 10 similar books
        book_indices = [i[0] for i in sim_scores]
        selected_books = data.iloc[book_indices]

        print("Top similar books and their indices:")
        for idx, score in sim_scores[:10]:
            print("Index:", idx, "Score:", score)

        print('selected_books')
        print(selected_books)
        
        return selected_books
    else:
        print(f"ReviewerID {reviewerID} not found in the training data.")
        return None



def evaluate(reviewerID, data, vectorizer, cosine_similarities):
    predicted_books = predict_ratings(reviewerID, train_data, vectorizer, cosine_similarities)
    true_ratings = val_data[val_data['reviewerID'] == reviewerID]['rating']
    predicted_ratings = predicted_books['rating']
    print("Length of true_ratings:", len(true_ratings))
    print("Length of predicted_ratings:", len(predicted_ratings))
    print("True ratings:", true_ratings)
    print("Present in the predicted ratings:", predicted_ratings)
    mae = mean_absolute_error(true_ratings, predicted_ratings)
    rmse = mean_squared_error(true_ratings, predicted_ratings, squared=False)
    return mae, rmse


# Example usage:
mae, rmse = evaluate(reviewerID, val_data, vectorizer, val_cosine_similarities)
print(f'MAE: {mae}, RMSE: {rmse}')


KeyError: 'combined'